# Result Visualisation#
## Imports
We start by importing some modules we need

In [24]:
import torch
import timm
from PIL import Image
from torchvision import tv_tensors
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from transformers import AutoImageProcessor
import importlib
from transformers import( Mask2FormerForUniversalSegmentation, Mask2FormerImageProcessor)
from transformers.models.mask2former.configuration_mask2former import Mask2FormerConfig
import pytorch_lightning 
from models.mask2former import Mask2FormerFinetuner
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import configs.runconfig as config
from  datastorage.cocodatamodule_semantic import CocoLightningDataModule_Semantic
from torchmetrics.segmentation import MeanIoU
from models.dinosemantic import DinoFinetuner

 ## Function Definition
Define Functions that we need for evaluation

### Semantic Segmentation
For Semantic Segemention we use mean intersection over union as a key metric

In [23]:
def evaluate_miou(model, dataloader, num_classes, device='cuda'):
    """
    Evaluate mean IoU over a test dataset using torchmetrics.segmentation.MeanIoU.
    """
    model.eval()
    model.to(device)

    metric = MeanIoU(num_classes=num_classes).to(device)

    with torch.no_grad():
        for batch in dataloader:
            # Adjust key names if necessary
            images = batch['image'].to(device)  
            masks = batch['mask'].to(device)    

            # Forward pass
            outputs = model(images)  # shape: (B, C, H, W)
            preds = torch.argmax(outputs, dim=1)  # class predictions (B, H, W)

            # Update metric
            metric.update(preds, masks)

    # Compute final mean IoU and per-class IoU
    results = metric.compute()  # returns dict with 'mean_iou' and 'per_class_iou'
    return results


## Evaluation

Here we evaluate a Dinov3 Model with a trained segmentaion head


In [ ]:
model = DinoFinetuner.load_from_checkpoint